<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [41]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-11-16 20:28:48--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz.2’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==> SIZE spark-3.3.1-bin-hadoop2.tgz ... 274099817
==> PASV ... done.    ==> RETR spark-3.3.1-bin-hadoop2.tgz ... done.
Length: 274099817 (261M) (unauthoritative)

spark-3.3.1-bin-had 100%[===================>] 261.40M  21.3MB/s    in 13s     

2022-11-16 20:29:02 (20.7 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz.2’ saved [274099817]



In [42]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [43]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [44]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-16 20:29:07--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 54.230.18.35, 54.230.18.48, 54.230.18.84, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|54.230.18.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236436122 (225M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz.2’

name.basics.tsv.gz. 100%[===================>] 225.48M  75.7MB/s    in 3.0s    

2022-11-16 20:29:10 (75.7 MB/s) - ‘name.basics.tsv.gz.2’ saved [236436122/236436122]

--2022-11-16 20:29:10--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 54.230.18.35, 54.230.18.48, 54.230.18.84, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|54.230.18.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287598573 (274M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz.2’

title.akas.tsv.gz.2 100%[===================>] 274.27M 

##Rozpakowanie danych

In [45]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [46]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [47]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00718...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00564...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [48]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [49]:
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw różnicę w różnych typach łączenia tabel df_name_basics_selected i df_title_principals_selected:

In [50]:
from timeit import default_timer as timer

start = timer()
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
end = timer()
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20762844, 8) time of joining tables:  0.019199426998966374


In [51]:
start = timer()
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "left")
end = timer()
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (30547803, 8) time of joining tables:  0.02596094799991988


In [52]:
start = timer()
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "outer")
end = timer()
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (30551431, 8) time of joining tables:  0.016509987000972615


In [53]:
start = timer()
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
end = timer()
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)), "time of joining tables: ", end-start)

df_name_basics_selected and df_title_principals_selected joined table size:  (20766472, 8) time of joining tables:  0.018478152000170667


In [ ]:
data1.filter(data1.primaryName.isNull()).show()
data2.filter(data2.primaryName.isNull()).show()
data3.filter(data3.primaryName.isNull()).show()
data4.filter(data4.primaryName.isNull()).show()

data1.filter(data1.tconst.isNull()).show()
data2.filter(data2.tconst.isNull()).show()
data3.filter(data3.tconst.isNull()).show()
data4.filter(data4.tconst.isNull()).show()

+------+-----------+---------+---------+--------------+------+--------+----------+
|nconst|primaryName|birthYear|deathYear|knownForTitles|tconst|category|characters|
+------+-----------+---------+---------+--------------+------+--------+----------+
+------+-----------+---------+---------+--------------+------+--------+----------+

+------+-----------+---------+---------+--------------+------+--------+----------+
|nconst|primaryName|birthYear|deathYear|knownForTitles|tconst|category|characters|
+------+-----------+---------+---------+--------------+------+--------+----------+
+------+-----------+---------+---------+--------------+------+--------+----------+

+---------+-----------+---------+---------+--------------+----------+--------+--------------------+
|   nconst|primaryName|birthYear|deathYear|knownForTitles|    tconst|category|          characters|
+---------+-----------+---------+---------+--------------+----------+--------+--------------------+
|nm0635557|       null|     null| 

Widzimy, że najszybszy rodzaj łączenia tabel to w tym przypadku "inner". Wiemy, że ten rodzaj łączenia nie gubi istotnych dla nas wierszy, gdyż osoby zawarte w takiej tabeli na pewno istnieją w tabeli df_name_basics_selected, czyli są znane z imienia i nazwiska oraz są aktorami i grali w jakimś filmie, co wynika z tabeli df_title_principals_selected.

Teraz zobaczymy, czy dołączając tabelę df_name_basics_selected do tabeli df_title_principals_selected dostaniemy lepszy wynik niż powyżej.

In [ ]:
start = timer()
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
end = timer()
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
end = timer()
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "outer")
end = timer()
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)), "time of joining tables: ", end-start)

In [ ]:
start = timer()
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "right")
end = timer()
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)), "time of joining tables: ", end-start)

In [ ]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample